In [3]:
#Set up the notebook to be able to fetch from the google bucket
import os

BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET']

In [86]:
#Copy files from the google bucket to the notebook's VM
!gcloud storage cp $bucket/* .

Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/COSMIC_noncoding_MCF7.tsv to file://./COSMIC_noncoding_MCF7.tsv
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/CRISPR_DepMap_Chronos.tsv to file://./CRISPR_DepMap_Chronos.tsv
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/CRISPR_DepMap_Chronos_MCF7.tsv to file://./CRISPR_DepMap_Chronos_MCF7.tsv
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/CRISPR_DepMap_DEMETER.tsv to file://./CRISPR_DepMap_DEMETER.tsv
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/EFO_0000305_associations_export.tsv to file://./EFO_0000305_associations_export.tsv
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/OmicsSomaticMutations.csv to file://./OmicsSomaticMutations.csv
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/domains.bedpe to file://./domains.bedpe
Copying gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/loops.bedpe to file://./loops.bedpe
  Completed files 8/8 | 872.6MiB/872.6MiB | 128.2MiB/s                         

Avera

In [60]:
import csv
from collections import defaultdict


"""
Transforms a tsv file into a dictionary of the form chromosome:row

Input:
file - a tsv that includes columns "#chr1" representing a chromosome, "x1" representing
a position on the chromosome and "x2" representing a position on the chromosome greater than
or equal to "x1"

Returns:
A dictionary with "#chr1" as the key and the input row as value, with values "x1" and "x2"
tranformed into ints
"""
def format_bedpe(file):            
    with open(file, "r") as f1:
        tsv_file = csv.DictReader(f1, delimiter="\t")
        dictified_tsv = defaultdict(list)
        count = 1
        for row in tsv_file:
            if row['x1']:
                row['x1'] = int(row['x1'])
                row['x2'] = int(row['x2'])
                dictified_tsv[row['#chr1']].append(row)
        return dictified_tsv


"""
Matches regions of a chromsome to regions that overlap in the input data

Input:
chrom - a chromosome in the format "chrNUM"
begin - a number representing position on the chromosome
end - a number representing position on the chromosome - should be greater than or equal to begin
input_data - a dictionary of the form chromosome:row, where row is a dictionary that includes 
keys "x1" - a number representing position on the chromsome and "x2" - a number representing
position on the chromsome, greater than or equal to x1

Returns:
A dictionary of the form gene:results, where results is a list of the rows from input_data that
are on the same chromosome of the gene and whose x-values overlap the gene's coordinates

"""
def get_match(chrom, begin, end, input_data):
    c1s = input_data[chrom]
    results = []
    for row in c1s:
        if not((begin <= row['x1'] and end <= row['x2']) or (begin >= row['x1'] and end >= row['x2'])):
            results.append(row)
    return results

In [61]:
#Import file
loops_dictified = format_bedpe("loops.bedpe")

#Define genes of interest

CRISPR = {
    "ESR1": ("chr6", 151656691, 152129619),
    "BCAS1": ("chr20", 53936777, 54070594),
    "BMP7": ("chr20", 57168753, 57266641),
    "FOXA1": ("chr14", 37589552, 37596059),
    "GATA3": ("chr10", 8045378, 8075198),
    "GABARAPL2": ("chr16", 75566375, 75577881),
    "TBX2": ("chr17", 61399843, 61409466),
    "TFAP2C": ("chr20", 56629306, 56639283),
    "NTNG1": ("chr1", 107140007, 107484923),
    "TCHHL1": ("chr1", 152084141, 152089064)
}

RNAi = {
    "MDM4": ("chr1", 204516379, 204558120),
    "ESR1": ("chr6", 151656691, 152129619),
    "KDM1A": ("chr1", 23019443, 23083689),
    "SAFB": ("chr19", 5623035, 5668478),
    "MDM2": ("chr12", 68808177, 68845544),
    "CCND1": ("chr11", 69641156, 69654474),
    "CDK4": ("chr12", 57747727, 57756013),
    "STK24": ("chr13", 98445185, 98577940),
    "CYP2A6": ("chr19", 40843541, 40850447),
    "RPS16": ("chr19", 39433137, 39435949)
}

In [62]:
#Check genes for matches in the loops file
crispr_results = CRISPR.copy()
for gene, coords in CRISPR.items():
    crispr_results[gene] = get_match(coords[0], coords[1], coords[2], loops_dictified)

RNAi_results = RNAi.copy()
for gene, coords in RNAi.items():
    RNAi_results[gene] = get_match(coords[0], coords[1], coords[2], loops_dictified)

In [63]:
"""
Transforms a tsv into a dictionary keyed by chromosome

Input:
association_file - a tsv file that contains a column "locations" with format
#chromosome:location

Returns
a dictionary with the chromosome as the key and the entire row as the value
"""
def group_associations(association_file):
    with open(association_file, "r") as file:
        association_tsv = csv.DictReader(file, delimiter="\t")
        association_dict = defaultdict(list)
        for line in association_tsv:
            location = line['locations'].split(":")
            chrom = location[0]
            association_dict[chrom].append(line)
    return association_dict


"""
Finds matches between ranges and a given location

Input:
associations - a dictionary of the form chromosome:associations, where chromosome is a number
and associations is list of dictionaries that include a key "locations" with value of the form "chromosome:position"
ex: # {1: [{riskAllele: rs123, locations: 1:123, ...}], 2: {riskAllele: rs456, locations: 2:456, ...}}
genes - a dictionary of the form gene:loops, where loops is a list of dictionaries that
include: key "chr2"/value chromosome of the form "chrNUM", key "y1"/value position, key "y2"/value position
ex: {ESR1: [{chr2: chr6, 'y1': '151','y2': '152',...}, {chr2: chr6, 'y1': '153','y2': '157',...}], 'MDM4': [...]}

Returns:
A dictionary with genes as keys and a list of tuples as values. 
The tuples include the data from the associations in the first spot, and data from genes in the second
"""
def cross_reference_associations(associations, genes):
    assoc_results = defaultdict(list)
    for chrom, locs in associations.items():
        for gene, results in genes.items():
            #chrom: 1
            #locs: [{..locations: 1:123},{..}]
            #gene: ESR1
            #results: [{chr2: chr6, 'y1': '151','y2': '152',...}, {chr2: chr6, 'y1': '153','y2': '157',...},...]
            for result in results:
                if result['chr2'].split('chr')[1] == chrom:
                    for loc in locs:
                        try:
                            location = int(loc['locations'].split(":")[1])
                            if location >= int(result['y1']) and location <= int(result['y2']):
                                assoc_results[gene].append((result,locs))
                        except:
                            pass
    return assoc_results        


In [82]:
"""
Turns a messy pile of data into readable results that we care about

Input:
results - A dictionary with genes as keys and a list of tuples as values. 
The tuples include the data from the associations in the first spot, and data from genes in the second

Returns:
A dictionary with genes as keys and a list of dictionaries as values.
The dictionaries include chromosome and position, riskAllele, pValue
"""
def massage_results(results):
    end_dict = defaultdict(list)
    for gene, tups in results.items():
        for tup in tups:
            for assoc in tup[1]:
                end_dict[gene].append({'location': assoc['locations'], 'riskAllele': assoc['riskAllele'], 'pValue': assoc['pValue']})
    return end_dict

In [83]:
#See if any of the locations in the assoc_file are contained in the y's of the
#results from above

assoc_file = "EFO_0000305_associations_export.tsv"
assoc_dict = group_associations(assoc_file)

crispr_assoc = cross_reference_associations(assoc_dict, crispr_results)
rnai_assoc = cross_reference_associations(assoc_dict, RNAi_results)

In [84]:
crispr_end = massage_results(crispr_assoc)
rnai_end = massage_results(rnai_assoc)

merged_results = rnai_end | crispr_end

In [85]:
merged_results

defaultdict(list,
            {'ESR1': [{'location': '6:151621059',
               'riskAllele': 'rs3757322-G',
               'pValue': '6E-9'},
              {'location': '6:151631197',
               'riskAllele': 'rs9397437-A',
               'pValue': '5E-9'},
              {'location': '6:151666222',
               'riskAllele': 'rs9383938-T',
               'pValue': '3E-9'},
              {'location': '6:33272092',
               'riskAllele': 'rs17215231-C',
               'pValue': '9E-13'},
              {'location': '6:151592978',
               'riskAllele': 'rs3757318-A',
               'pValue': '9E-6'},
              {'location': '6:135969804',
               'riskAllele': 'rs11154838-C',
               'pValue': '1E-6'},
              {'location': '6:151623479',
               'riskAllele': 'rs9383936-A',
               'pValue': '1E-10'},
              {'location': '6:151627231',
               'riskAllele': 'rs2046210-A',
               'pValue': '1E-9'},
           

In [144]:
merged_results_file = open("riskAlleles.txt", "a")
for gene, results in merged_results.items():
#     merged_results_file.write(gene)
#     merged_results_file.write(str(results))
    merged_results_file.write(gene)
    for result in results:
        merged_results_file.write(str(result))
merged_results_file.close()

In [145]:
!gcloud storage cp "riskAlleles.txt" $bucket

Copying file://riskAlleles.txt to gs://fc-6432ab48-f392-48b4-bfc0-43d3f3ad482e/riskAlleles.txt
  Completed files 1/1 | 90.4kiB/90.4kiB                                        


In [132]:
"""
Finds matches on position and notes whether riskAllele = genomic mut allele

Input:
filtered_results - A dictionary with genes as keys and a list of dictionaries as values.
The dictionaries include chromosome and position, riskAllele, pValue
mutations - a list of dictionaries that include columns "CHROMSOME", "GENOME_START", "GENOME_STOP" and "GENOMIC_MUT_ALLELE"

Returns:
A dictionary with genes as keys and values that are lists of tuples of the form
(match, result, mut) where match is True/False indicating whether the riskAllele from the filtered results
matches the genomic mutation allele, result is the entire row from the filtered results,
and mut is the entire row from the mutations list
"""
def match_allele(filtered_results, mutations):
    matches = defaultdict(list)
    for gene, results in filtered_results.items():
        for result in results:
            for mut in mutations:
                locations = result['location'].split(",")
                for loc in locations:
                    chrom, pos = loc.split(':')
                    if chrom == mut['CHROMOSOME'] and (pos == mut['GENOME_START'] or pos == mut['GENOME_STOP']):
                        print("match found")
                        allele = result['riskAllele'].split('-')[1]
                        match = allele != '?' and allele == mut['GENOMIC_MUT_ALLELE']
                        matches[gene].append((match, result, mut))
    return matches
    

In [150]:
file = "COSMIC_noncoding_MCF7.tsv"

def transform_mutations(file):
    mutations = []
    with open(file, "r") as f1:
        mutation_tsv = csv.DictReader(f1, delimiter="\t")
        for line in mutation_tsv:
            mutations.append(line)
    return mutations

In [133]:
matches = match_allele(merged_results, mutations)

In [129]:
matches.items()

dict_items([])

In [148]:
domains_dictified = format_bedpe("domains.bedpe")
crispr_results2 = CRISPR.copy()
for gene, coords in CRISPR.items():
    crispr_results2[gene] = get_match(coords[0], coords[1], coords[2], domains_dictified)

RNAi_results2 = RNAi.copy()
for gene, coords in RNAi.items():
    RNAi_results2[gene] = get_match(coords[0], coords[1], coords[2], domains_dictified)

crispr_assoc2 = cross_reference_associations(assoc_dict, crispr_results2)
rnai_assoc2 = cross_reference_associations(assoc_dict, RNAi_results2)

crispr_end2 = massage_results(crispr_assoc2)
rnai_end2 = massage_results(rnai_assoc2)

merged_results2 = rnai_end2 | crispr_end2

matches2 = match_allele(merged_results2, mutations)

In [151]:
"""
Inputs:
contact domains - file identifying locations of high interaction
gene positions - genes of interest and their positions
associations file - regions of interest
noncoding file - more regions to match
"""
def pipeline(contact_domains, gene_pos, associations_file, noncoding_file):
    domains = format_bedpe(contact_domains)
    gene_results = gene_pos.copy()
    for gene, coords in gene_pos.items():
        gene_results[gene] = get_match(coords[0], coords[1], coords[2], domains)
    
    assoc_dict = group_associations(associations_file)
    gene_assoc = cross_reference_associations(assoc_dict, gene_results)
    
    gene_assoc_reformat = massage_results(gene_assoc)
    
    mutations = transform_mutations(noncoding_file)

    matches = match_allele(gene_assoc_reformat, mutations)
    
    return matches
    

In [152]:
pipeline("domains.bedpe", RNAi, "EFO_0000305_associations_export.tsv", "COSMIC_noncoding_MCF7.tsv")

defaultdict(list, {})